# Ch11. 텍스트 분류 (Text Classification)

# v04. IMDB 리뷰 감성 분류하기 (IMDB Movie Review Sentiment Analysis)

- 머신 러닝에서 텍스트 분류, 그 중에서도 특히 감성 분류를 연습하기 위해 자주 사용하는 데이터가 있다.
- 바로 영화 사이트 IMDB의 리뷰 데이터이다.
- 이 데이터는 리뷰에 대한 텍스트와 해당 리뷰가 긍정인 경우 1을 부정인 경우 0으로 표시한 레이블로 구성된 데이터이다.

- 스탠포드 대학교에서 2011년에 낸 논문에서 이 데이터를 소개하였다.
- 당시 논문에서는 이 데이터를 훈련 데이터와 테스트 데이터를 50:50 비율로 분할하여 88.89%의 정확도를 얻었다고 소개하고 있다.
- [논문 링크](http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf)

- 케라스에서는 해당 IMDB 영화 리뷰 데이터를 `imdb.load_data()` 함수를 통해 다운로드 할 수 있도록 지원하고 있다.
- 케라스로부터 해당 데이터를 다운로드 받아 감성 분류를 수행하는 모델을 만들어보자.

<br>

## 4.1 IMDB 리뷰 데이터에 대한 이해

### 4.1.1 필요 라이브러리 임포트

In [ ]:
from tensorflow.keras.datasets import imdb
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

<br>

### 4.1.2 데이터 불러오기

- 영화 리뷰 데이터를 갖고오자.
- 케라스에서 제공하는 IMDB 리뷰 데이터는 앞서 배운 로이터 뉴스 데이터에서 훈련 데이터와 테스트 데이터를 우리가 직접 비율을 조절했던 것과는 달리 이미 훈련 데이터와 테스트 데이터를 50:50 비율로 구분해서 제공한다.
- 로이터 뉴스 데이터에서 사용했던 `test_split`과 같은 데이터의 비율을 조절하는 파라미터는 `imdb.load_data`에서는 지원하지 않는다.

In [2]:
# 영화 리뷰는 X_train에, 감성 정보는 y_train에 저장된다.
# 테스트용 리뷰는 X_test에, 테스트용 리뷰의 감성 정보는 y_test에 저장된다.
(X_train, y_train), (X_test, y_test) = imdb.load_data()
# (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 0s 0us/step


- `imdb.data_load()`의 파라미터로 `num_words`를 사용하면 이 데이터에서 등장 빈도 순위로 몇 번째에 해당하는 단어까지를 사용할 것인지를 의미한다.
- 예를 들어서 `10000`을 넣으면, 등장 빈도 순위가 1~10,000에 해당하는 단어만 사용하게 된다.
- 즉, 단어 집합의 크기는 10,000이 된다.
- 지금은 별도로 제한하지 않는다.

<br>

### 4.1.3 데이터 개수 확인

In [3]:
print('훈련용 리뷰 개수 : {}'.format(len(X_train)))
print('테스트용 리뷰 개수 : {}'.format(len(X_test )))

훈련용 리뷰 개수 : 25000
테스트용 리뷰 개수 : 25000


In [4]:
num_classes = max(y_train) + 1
print('카테고리 : {}'.format(num_classes))

카테고리 : 2


- `y_train`는 0부터 시작해서 레이블을 부여한다.
- 그러므로 `y_train`에 들어 있는 가장 큰 수에 +1을 하여 출력하면 카테고리가 총 몇 개인 지를 알 수 있다.

- 훈련용 리뷰는 25,000개, 테스트용 리뷰는 25,000개, 카테고리는 2개이다.

<br>

### 4.1.4 훈련 데이터의 구성 확인

- 훈련 데이터가 어떻게 구성되어 있는 지를 확인하기 위해 첫 번째 훈련용 리뷰를 출력해보자.
- 다시 말해, 25,000개의 영화 리뷰 중 첫 번째 리뷰 텍스트와 그 리뷰에 대한 레이블을 출력해보는 것이다.

In [7]:
print(X_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


- 첫 번째 훈련용 리뷰(`X_train[0]`, `y_train[0]`)에서 리뷰 본문에 해당하는 `X_train[0]`에는 숫자들이 들어 있다.
- 이 데이터는 토큰화와 정수 인코딩이라는 텍스트 전처리가 끝난 상태이다.

- IMDB 리뷰 데이터는 전체 데이터에서 각 단어들의 등장 빈도에 따라서 인덱스를 부여했다.
  - `1`이라는 숫자는 이 단어가 이 데이터에서 등장 빈도가 1등이라는 뜻이다.
  - `973`이라는 숫자는 이 단어가 데이터에서 973번 째로 빈도수가 높은 단어라는 뜻이다.  


- 그리고 위에서 단어 집합의 크기를 제한하지 않았기 때문에 22,665와 같은 큰 숫자도 보인다.

In [8]:
print(y_train[0])

1


- 첫 번째 훈련용 리뷰의 